stock_data RE

In [ ]:
import pandas as pd
import numpy as np

# Define the set of relevant tickers for the project's scope
# I've included a few common meme/tech stocks for context
TICKERS_TO_ANALYZE = ['GME', 'AMC', 'BBBY', 'TSLA', 'AAPL', 'AMZN']

# --- Phase 1: Stock Data ETL (Base DataFrame Creation) ---

# 1. Load Stock Data (Task 3)
stock_df = pd.read_csv('Clean_Task3_StockData.csv')

# 2. Convert date to datetime
stock_df['date'] = pd.to_datetime(stock_df['date'])

# Filter for the relevant tickers
stock_df = stock_df[stock_df['ticker'].isin(TICKERS_TO_ANALYZE)]

# Sort the data by ticker and date. This is CRUCIAL for correct pct_change calculation.
stock_df.sort_values(by=['ticker', 'date'], inplace=True)

# 3. Group by ticker and calculate the target variable: price_change_pct
# pct_change() calculates (Current_Price - Previous_Price) / Previous_Price
stock_df['price_change_pct'] = stock_df.groupby('ticker')['close'].pct_change()

# 4. Create the final base DataFrame with only the required columns
# The 'close' price is also included for potential future normalization or visualization
base_df = stock_df[['date', 'ticker', 'price_change_pct', 'volume', 'close']].copy()

# Drop the initial NaN row for each ticker (the first day has no previous price to compare to)
base_df.dropna(subset=['price_change_pct'], inplace=True)

# Reset index and print a summary
base_df.reset_index(drop=True, inplace=True)

# --- Save for Next Steps ---
base_df.to_csv('Base_Stock_DataFrame.csv', index=False)
print("Base Stock DataFrame created and saved to 'Base_Stock_DataFrame.csv'.")
print("\nFirst 5 rows of the base data:")
print(base_df.head())
print("\nDataFrame structure:")
base_df.info()

Base Stock DataFrame created and saved to 'Base_Stock_DataFrame.csv'.

First 5 rows of the base data:
        date ticker  price_change_pct      volume       close
0 2024-05-16   AAPL          0.006173  52393600.0  186.346039
1 2024-05-17   AAPL          0.012218  70400000.0  188.622803
2 2024-05-20   AAPL          0.000632  52845200.0  188.742096
3 2024-05-21   AAPL          0.000158  41282900.0  188.771927
4 2024-05-22   AAPL          0.006162  44361300.0  189.935150

DataFrame structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              464 non-null    datetime64[ns]
 1   ticker            464 non-null    object        
 2   price_change_pct  464 non-null    float64       
 3   volume            464 non-null    float64       
 4   close             464 non-null    float64       
dtypes: datetime64[ns]

In [2]:

finmemes_df = pd.read_csv("Clean_Task2_FinMemes.csv")
base_df = pd.read_csv("Base_Stock_DataFrame.csv")

# Quick look at data
print("FinMemes columns:", finmemes_df.columns.tolist())
print("Base Data columns:", base_df.columns.tolist())
finmemes_df.head()

FinMemes columns: ['subreddit', 'month_collected', 'id', 'title', 'selftext', 'url', 'score', 'num_comments', 'created_dt', 'collection_type', 'tickers_mentioned']
Base Data columns: ['date', 'ticker', 'price_change_pct', 'volume', 'close']


,subreddit,month_collected,id,title,selftext,url,score,num_comments,created_dt,collection_type,tickers_mentioned
0,wallstreetbets,2024-10,1gfxoui,Cooked,NaN,https://i.redd.it/gsbduo4lnyxd1.jpeg,47263,192,2024-10-30 21:21:53+00:00,top,NONE
1,wallstreetbets,2024-12,1hhicdz,Are you shitting me rn,NaN,https://i.redd.it/xtlidm9ksp7e1.jpeg,47155,919,2024-12-19 02:15:05+00:00,top,NONE
2,wallstreetbets,2024-12,1h6to8b,"My Uber driver just checked his portfolio, sho...",NaN,https://v.redd.it/p0wzfdqruw4e1,42741,1020,2024-12-04 22:48:02+00:00,top,NONE
3,wallstreetbets,2024-12,1h6leat,"""CEO gets gunned down in the street outside an...",NaN,https://i.redd.it/nrbbjo7d7v4e1.png,36685,2217,2024-12-04 17:15:13+00:00,top,NONE
4,wallstreetbets,2024-11,1gm03j1,"JPow gave 'em the ""I'm not fucking leaving""",NaN,https://v.redd.it/4s1hq268gjzd1,31556,1716,2024-11-07 20:22:15+00:00,top,NONE


In [3]:
finmemes_df['tickers_mentioned'] = finmemes_df['tickers_mentioned'].replace('NONE', '').fillna('')

# Split multiple tickers like 'BTC, ETH' into a list ['BTC', 'ETH']
finmemes_df['tickers_mentioned'] = finmemes_df['tickers_mentioned'].apply(
    lambda x: [t.strip() for t in x.split(',') if t.strip()] if x else []
)

# Verify parsing
finmemes_df[['tickers_mentioned']].head(10)

,tickers_mentioned
0,[]
1,[]
2,[]
3,[]
4,[]
5,[]
6,[]
7,[]
8,[]
9,[]


Remove rows with zero activity

In [4]:
# Expand so that each ticker gets its own row
exploded_df = finmemes_df.explode('tickers_mentioned')

# Drop rows without tickers
exploded_df = exploded_df[exploded_df['tickers_mentioned'] != '']

# Create a clean 'date' column
exploded_df['date'] = pd.to_datetime(exploded_df['created_dt']).dt.date

exploded_df.head()

,subreddit,month_collected,id,title,selftext,url,score,num_comments,created_dt,collection_type,tickers_mentioned,date
0,wallstreetbets,2024-10,1gfxoui,Cooked,NaN,https://i.redd.it/gsbduo4lnyxd1.jpeg,47263,192,2024-10-30 21:21:53+00:00,top,NaN,2024-10-30
1,wallstreetbets,2024-12,1hhicdz,Are you shitting me rn,NaN,https://i.redd.it/xtlidm9ksp7e1.jpeg,47155,919,2024-12-19 02:15:05+00:00,top,NaN,2024-12-19
2,wallstreetbets,2024-12,1h6to8b,"My Uber driver just checked his portfolio, sho...",NaN,https://v.redd.it/p0wzfdqruw4e1,42741,1020,2024-12-04 22:48:02+00:00,top,NaN,2024-12-04
3,wallstreetbets,2024-12,1h6leat,"""CEO gets gunned down in the street outside an...",NaN,https://i.redd.it/nrbbjo7d7v4e1.png,36685,2217,2024-12-04 17:15:13+00:00,top,NaN,2024-12-04
4,wallstreetbets,2024-11,1gm03j1,"JPow gave 'em the ""I'm not fucking leaving""",NaN,https://v.redd.it/4s1hq268gjzd1,31556,1716,2024-11-07 20:22:15+00:00,top,NaN,2024-11-07


In [5]:
# Use Reddit 'score' as a simple sentiment proxy for now
exploded_df['wsb_sentiment'] = exploded_df['score']

# Group by date and ticker
agg_df = (
    exploded_df.groupby(['date', 'tickers_mentioned'])
    .agg(
        wsb_sentiment=('wsb_sentiment', 'mean'),
        wsb_post_count=('id', 'count')
    )
    .reset_index()
    .rename(columns={'tickers_mentioned': 'ticker'})
)

agg_df.head(15)


,date,ticker,wsb_sentiment,wsb_post_count
0,2024-01-07,AVAX,1.0,1
1,2024-01-07,DEX,1.0,1
2,2024-01-07,PS,1.0,1
3,2024-01-07,USD,1.0,1
4,2024-01-08,ALT,2.0,1
5,2024-01-08,AVAX,2.0,1
6,2024-01-08,BTC,2.5,2
7,2024-01-08,CEX,1.0,1
8,2024-01-08,ETH,2.5,2
9,2024-01-08,HKD,3.0,1


In [6]:
# Get unique tickers from both datasets
finmemes_tickers = set(agg_df['ticker'].unique())
base_tickers = set(base_df['ticker'].unique())

# Union (all unique tickers from both)
all_tickers = finmemes_tickers.union(base_tickers)

print(f"✅ Total unique tickers across both datasets: {len(all_tickers)}")
print(list(all_tickers)[:30])  # Preview first 30 tickers


✅ Total unique tickers across both datasets: 722
['MODEL', 'SAND', 'LIVE', 'RON', 'DAILY', 'MODAI', 'RARE', 'VI', 'NW', 'PCAOB', 'SOUTH', 'VIP', 'PER', 'HELOC', 'TFSA', 'BXN', 'DCA', 'VPM', 'DIY', 'KAIJU', 'PEPEO', 'ESPP', 'STATS', 'KO', 'TIPS', 'LSD', 'OSMO', 'HEGE', 'LOT', 'ALMC']


In [7]:
# Ensure 'date' column is of same type
base_df['date'] = pd.to_datetime(base_df['date']).dt.date

# Merge Reddit features into base stock data
merged_df = base_df.merge(agg_df, on=['date', 'ticker'], how='left')

merged_df.head(15)


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count
0,2024-05-16,AAPL,0.006173,52393600.0,186.346039,NaN,NaN
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803,NaN,NaN
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096,NaN,NaN
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927,NaN,NaN
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150,NaN,NaN
5,2024-05-23,AAPL,0.006857,42309400.0,191.237595,NaN,NaN
6,2024-05-24,AAPL,-0.007538,34648500.0,189.795959,NaN,NaN
7,2024-05-28,AAPL,-0.021058,51005900.0,185.799225,NaN,NaN
8,2024-05-29,AAPL,0.016588,36327000.0,188.881287,NaN,NaN
9,2024-05-30,AAPL,0.000053,52280100.0,188.891235,NaN,NaN


In [8]:
# Fill missing post counts with 0
merged_df['wsb_post_count'] = merged_df['wsb_post_count'].fillna(0)

# Smooth missing sentiment values using a rolling mean per ticker
merged_df['wsb_sentiment'] = (
    merged_df.groupby('ticker')['wsb_sentiment']
    .transform(lambda x: x.fillna(x.rolling(3, min_periods=1).mean()))
)

# Optional: replace any remaining NaNs with 0
merged_df['wsb_sentiment'] = merged_df['wsb_sentiment'].fillna(0)

merged_df.head(15)


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count
0,2024-05-16,AAPL,0.006173,52393600.0,186.346039,0.0,0.0
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803,0.0,0.0
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096,0.0,0.0
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927,0.0,0.0
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150,0.0,0.0
5,2024-05-23,AAPL,0.006857,42309400.0,191.237595,0.0,0.0
6,2024-05-24,AAPL,-0.007538,34648500.0,189.795959,0.0,0.0
7,2024-05-28,AAPL,-0.021058,51005900.0,185.799225,0.0,0.0
8,2024-05-29,AAPL,0.016588,36327000.0,188.881287,0.0,0.0
9,2024-05-30,AAPL,0.000053,52280100.0,188.891235,0.0,0.0


In [9]:
merged_df.to_csv("Merged_WSB_Stock_Data_1.csv", index=False)
print(" Merged file saved as Merged_WSB_Stock_Data.csv")


 Merged file saved as Merged_WSB_Stock_Data.csv


In [10]:
print("🧾 Summary of merged data:")
print(merged_df.info())

# Count non-zero post counts
non_zero_posts = (merged_df['wsb_post_count'] > 0).sum()
print(f"📊 Rows with at least one Reddit post: {non_zero_posts}")

# Check ticker distribution
merged_df['ticker'].value_counts().head(20)


🧾 Summary of merged data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              464 non-null    object 
 1   ticker            464 non-null    object 
 2   price_change_pct  464 non-null    float64
 3   volume            464 non-null    float64
 4   close             464 non-null    float64
 5   wsb_sentiment     464 non-null    float64
 6   wsb_post_count    464 non-null    float64
dtypes: float64(5), object(2)
memory usage: 25.5+ KB
None
📊 Rows with at least one Reddit post: 4


ticker
AAPL    116
AMZN    116
GME     116
TSLA    116
Name: count, dtype: int64

phase-2 temporal lag creation

Phase3

In [12]:
import os

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

def df_info(df, name=None, n=5):
    print(f"--- {name or 'DF'} shape: {df.shape} ---")
    display(df.head(n))
    print(df.dtypes)
    print(df.isna().sum())
    print()

In [14]:

base_path = 'Base_Stock_DataFrame.csv'                       # stock base (Phase1 output)
wsb_correct_path = 'Merged_WSB_Stock_Data.csv'              # correct merged (use this)
general_meme_path = 'Clean_Task1_GeneralMeme_Aggregated.csv'  # aggregated by date

base = pd.read_csv(base_path)
wsb = pd.read_csv(wsb_correct_path)
meme = pd.read_csv(general_meme_path)

df_info(base, 'base (stock)')
df_info(wsb, 'wsb (merged correct)')
df_info(meme, 'general_meme')


--- base (stock) shape: (464, 5) ---


,date,ticker,price_change_pct,volume,close
0,2024-05-16,AAPL,0.006173,52393600.0,186.346039
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150


date                 object
ticker               object
price_change_pct    float64
volume              float64
close               float64
dtype: object
date                0
ticker              0
price_change_pct    0
volume              0
close               0
dtype: int64

--- wsb (merged correct) shape: (464, 7) ---


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count
0,2024-05-16,AAPL,0.006173,52393600.0,186.346039,NaN,0.0
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803,NaN,0.0
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096,NaN,0.0
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927,NaN,0.0
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150,NaN,0.0


date                 object
ticker               object
price_change_pct    float64
volume              float64
close               float64
wsb_sentiment       float64
wsb_post_count      float64
dtype: object
date                  0
ticker                0
price_change_pct      0
volume                0
close                 0
wsb_sentiment       464
wsb_post_count        0
dtype: int64

--- general_meme shape: (225, 3) ---


,date,general_meme_virality,meme_post_count
0,2024-03-02 00:00:00+00:00,13,1
1,2024-03-03 00:00:00+00:00,30,1
2,2024-03-04 00:00:00+00:00,58,1
3,2024-03-06 00:00:00+00:00,79,2
4,2024-03-07 00:00:00+00:00,1435,2


date                     object
general_meme_virality     int64
meme_post_count           int64
dtype: object
date                     0
general_meme_virality    0
meme_post_count          0
dtype: int64



In [16]:
print("Unique tickers - base:", base['ticker'].nunique())
print("Unique tickers - wsb:", wsb['ticker'].nunique())
print("Unique tickers - meme: (should be NaN or broadcasted)", meme.get('ticker', pd.Series()).nunique() if 'ticker' in meme.columns else 'no ticker col')

# duplicates on (date,ticker)
for name, df in [('base', base), ('wsb', wsb), ('meme', meme)]:
    if set(['date','ticker']).issubset(df.columns):
        dup = df.duplicated(subset=['date','ticker']).sum()
        print(f"{name} duplicates (date,ticker): {dup}")
    else:
        print(f"{name} has no (date,ticker) columns to check dupes")

# row difference between union & correct merge
print( "correct merged rows:", wsb.shape)

Unique tickers - base: 4
Unique tickers - wsb: 4
Unique tickers - meme: (should be NaN or broadcasted) no ticker col
base duplicates (date,ticker): 0
wsb duplicates (date,ticker): 0
meme has no (date,ticker) columns to check dupes
correct merged rows: (464, 7)


In [17]:
def normalize_dates(df, date_col='date'):
    df[date_col] = pd.to_datetime(df[date_col], utc=True, errors='coerce')
    df[date_col] = df[date_col].dt.tz_convert(None)
    return df

base = normalize_dates(base, 'date')
wsb = normalize_dates(wsb, 'date')
meme = normalize_dates(meme, 'date')

# drop exact duplicates (all columns identical)
base = base.drop_duplicates()
wsb = wsb.drop_duplicates()
meme = meme.drop_duplicates()

df_info(base, 'base after date norm')
df_info(wsb, 'wsb after date norm')
df_info(meme, 'meme after date norm')

--- base after date norm shape: (464, 5) ---


,date,ticker,price_change_pct,volume,close
0,2024-05-16,AAPL,0.006173,52393600.0,186.346039
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150


date                datetime64[ns]
ticker                      object
price_change_pct           float64
volume                     float64
close                      float64
dtype: object
date                0
ticker              0
price_change_pct    0
volume              0
close               0
dtype: int64

--- wsb after date norm shape: (464, 7) ---


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count
0,2024-05-16,AAPL,0.006173,52393600.0,186.346039,NaN,0.0
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803,NaN,0.0
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096,NaN,0.0
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927,NaN,0.0
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150,NaN,0.0


date                datetime64[ns]
ticker                      object
price_change_pct           float64
volume                     float64
close                      float64
wsb_sentiment              float64
wsb_post_count             float64
dtype: object
date                  0
ticker                0
price_change_pct      0
volume                0
close                 0
wsb_sentiment       464
wsb_post_count        0
dtype: int64

--- meme after date norm shape: (225, 3) ---


,date,general_meme_virality,meme_post_count
0,2024-03-02,13,1
1,2024-03-03,30,1
2,2024-03-04,58,1
3,2024-03-06,79,2
4,2024-03-07,1435,2


date                     datetime64[ns]
general_meme_virality             int64
meme_post_count                   int64
dtype: object
date                     0
general_meme_virality    0
meme_post_count          0
dtype: int64



In [18]:
if base.duplicated(subset=['date','ticker']).any():
    print("WARN: base has duplicate (date,ticker). Aggregating by taking mean for numeric fields.")
    agg_cols = [c for c in base.columns if c not in ['date','ticker']]
    base = base.groupby(['date','ticker'], as_index=False)[agg_cols].mean()
else:
    print("base has unique (date,ticker). OK.")


base has unique (date,ticker). OK.


In [19]:
wsb_clean = wsb[['date', 'ticker', 'wsb_sentiment', 'wsb_post_count']]

df = base.merge(
    wsb_clean,
    on=['date', 'ticker'],
    how='left'                               # << KEEP STOCK ROWS ONLY
)

# Now merge meme data (broadcast across tickers)
df = df.merge(
    meme,
    on='date',
    how='left'
)

df_info(df, 'After correct merge (Phase 3 rebuild)')

--- After correct merge (Phase 3 rebuild) shape: (464, 9) ---


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count,general_meme_virality,meme_post_count
0,2024-05-16,AAPL,0.006173,52393600.0,186.346039,NaN,0.0,41269.0,11.0
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803,NaN,0.0,10217.0,16.0
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096,NaN,0.0,23484.0,10.0
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927,NaN,0.0,10181.0,10.0
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150,NaN,0.0,34714.0,16.0


date                     datetime64[ns]
ticker                           object
price_change_pct                float64
volume                          float64
close                           float64
wsb_sentiment                   float64
wsb_post_count                  float64
general_meme_virality           float64
meme_post_count                 float64
dtype: object
date                       0
ticker                     0
price_change_pct           0
volume                     0
close                      0
wsb_sentiment            464
wsb_post_count             0
general_meme_virality      8
meme_post_count            8
dtype: int64



In [20]:
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)

# Recompute returns to ensure accuracy
df['price_change_pct'] = df.groupby('ticker')['close'].pct_change()

df_info(df.head(20), 'After recomputing returns')

--- After recomputing returns shape: (20, 9) ---


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count,general_meme_virality,meme_post_count
0,2024-05-16,AAPL,NaN,52393600.0,186.346039,NaN,0.0,41269.0,11.0
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803,NaN,0.0,10217.0,16.0
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096,NaN,0.0,23484.0,10.0
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927,NaN,0.0,10181.0,10.0
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150,NaN,0.0,34714.0,16.0


date                     datetime64[ns]
ticker                           object
price_change_pct                float64
volume                          float64
close                           float64
wsb_sentiment                   float64
wsb_post_count                  float64
general_meme_virality           float64
meme_post_count                 float64
dtype: object
date                      0
ticker                    0
price_change_pct          1
volume                    0
close                     0
wsb_sentiment            20
wsb_post_count            0
general_meme_virality     0
meme_post_count           0
dtype: int64



In [21]:
# Cell 8: Fill missing WSB & general meme values

wsb_cols = ['wsb_sentiment', 'wsb_post_count']
meme_cols = ['general_meme_virality', 'meme_post_count']

df[wsb_cols] = df[wsb_cols].fillna(0)
df[meme_cols] = df[meme_cols].fillna(0)

df_info(df.head(10), 'After filling WSB & Meme NaNs with 0')


--- After filling WSB & Meme NaNs with 0 shape: (10, 9) ---


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count,general_meme_virality,meme_post_count
0,2024-05-16,AAPL,NaN,52393600.0,186.346039,0.0,0.0,41269.0,11.0
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803,0.0,0.0,10217.0,16.0
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096,0.0,0.0,23484.0,10.0
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927,0.0,0.0,10181.0,10.0
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150,0.0,0.0,34714.0,16.0


date                     datetime64[ns]
ticker                           object
price_change_pct                float64
volume                          float64
close                           float64
wsb_sentiment                   float64
wsb_post_count                  float64
general_meme_virality           float64
meme_post_count                 float64
dtype: object
date                     0
ticker                   0
price_change_pct         1
volume                   0
close                    0
wsb_sentiment            0
wsb_post_count           0
general_meme_virality    0
meme_post_count          0
dtype: int64



In [22]:
# Sort again just to be 100% safe
df = df.sort_values(['ticker', 'date']).reset_index(drop=True)

# 1-day lag for WSB posts
df['wsb_post_count_lag_1d'] = df.groupby('ticker')['wsb_post_count'].shift(1)

# 3-day lag for general meme virality
df['general_meme_virality_lag_3d'] = df.groupby('ticker')['general_meme_virality'].shift(3)

df_info(df.head(15), "After creating lag features")

--- After creating lag features shape: (15, 11) ---


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count,general_meme_virality,meme_post_count,wsb_post_count_lag_1d,general_meme_virality_lag_3d
0,2024-05-16,AAPL,NaN,52393600.0,186.346039,0.0,0.0,41269.0,11.0,NaN,NaN
1,2024-05-17,AAPL,0.012218,70400000.0,188.622803,0.0,0.0,10217.0,16.0,0.0,NaN
2,2024-05-20,AAPL,0.000632,52845200.0,188.742096,0.0,0.0,23484.0,10.0,0.0,NaN
3,2024-05-21,AAPL,0.000158,41282900.0,188.771927,0.0,0.0,10181.0,10.0,0.0,41269.0
4,2024-05-22,AAPL,0.006162,44361300.0,189.935150,0.0,0.0,34714.0,16.0,0.0,10217.0


date                            datetime64[ns]
ticker                                  object
price_change_pct                       float64
volume                                 float64
close                                  float64
wsb_sentiment                          float64
wsb_post_count                         float64
general_meme_virality                  float64
meme_post_count                        float64
wsb_post_count_lag_1d                  float64
general_meme_virality_lag_3d           float64
dtype: object
date                            0
ticker                          0
price_change_pct                1
volume                          0
close                           0
wsb_sentiment                   0
wsb_post_count                  0
general_meme_virality           0
meme_post_count                 0
wsb_post_count_lag_1d           1
general_meme_virality_lag_3d    3
dtype: int64



In [23]:
# Cell 10: Drop rows lacking lag history (Phase 3.5 final cleaning)

required_lags = ['wsb_post_count_lag_1d', 'general_meme_virality_lag_3d']

before = df.shape[0]

df_phase35 = df.dropna(subset=required_lags).reset_index(drop=True)

after = df_phase35.shape[0]

print(f"Dropped {before - after} rows that lacked lag history.")
df_info(df_phase35.head(15), 'Phase 3.5 final dataset (lag-cleaned)')


Dropped 12 rows that lacked lag history.
--- Phase 3.5 final dataset (lag-cleaned) shape: (15, 11) ---


,date,ticker,price_change_pct,volume,close,wsb_sentiment,wsb_post_count,general_meme_virality,meme_post_count,wsb_post_count_lag_1d,general_meme_virality_lag_3d
0,2024-05-21,AAPL,0.000158,41282900.0,188.771927,0.0,0.0,10181.0,10.0,0.0,41269.0
1,2024-05-22,AAPL,0.006162,44361300.0,189.935150,0.0,0.0,34714.0,16.0,0.0,10217.0
2,2024-05-23,AAPL,0.006857,42309400.0,191.237595,0.0,0.0,10465.0,13.0,0.0,23484.0
3,2024-05-24,AAPL,-0.007538,34648500.0,189.795959,0.0,0.0,46534.0,8.0,0.0,10181.0
4,2024-05-28,AAPL,-0.021058,51005900.0,185.799225,0.0,0.0,8320.0,7.0,0.0,34714.0


date                            datetime64[ns]
ticker                                  object
price_change_pct                       float64
volume                                 float64
close                                  float64
wsb_sentiment                          float64
wsb_post_count                         float64
general_meme_virality                  float64
meme_post_count                        float64
wsb_post_count_lag_1d                  float64
general_meme_virality_lag_3d           float64
dtype: object
date                            0
ticker                          0
price_change_pct                0
volume                          0
close                           0
wsb_sentiment                   0
wsb_post_count                  0
general_meme_virality           0
meme_post_count                 0
wsb_post_count_lag_1d           0
general_meme_virality_lag_3d    0
dtype: int64



In [24]:

df.to_csv('Phase3_Rebuilt.csv', index=False)
df_phase35.to_csv('Phase3_5_Rebuilt_Lagged.csv', index=False)

print("Saved Phase 3 → Phase3_Rebuilt.csv")
print("Saved Phase 3.5 → Phase3_5_Rebuilt_Lagged.csv")


Saved Phase 3 → Phase3_Rebuilt.csv
Saved Phase 3.5 → Phase3_5_Rebuilt_Lagged.csv


pre phase3 data check